In [1]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.style.use('fivethirtyeight')

import os

from tensorflow.examples.tutorials.mnist import input_data

C:\Users\aleksander.molak\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# GANs with TF

#### Some useful terms:


* **Xavier Initialization** https://prateekvjoshi.com/2016/03/29/understanding-xavier-initialization-in-deep-neural-networks/

    is a method of weights initialization to preserve the variance of input signal. 
    Original formula (http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf):
    
    $$var(w_i) = 1/N_{avg}$$
    
    where 
    
    $N_{avg} = \frac{(N_{in} + N_{out})}{2}$
    
    and $N$ is a number of nodes.
    
    Another often used formula:
    
    $$var(w_i) = 1/N_{in}$$

In [13]:
def xavier_init(size):
    input_dim = size[0]
    xavier_std = 1 / tf.sqrt(input_dim/2.)
    return tf.random_normal(shape=size, stddev=xavier_std)

In [66]:
# ADD: Eval tensors
sess = tf.InteractiveSession()
c = tf.random_normal(shape=(12,3), mean=100, stddev=15)
hhh = tf.zeros(128)
jjj = tf.zeros(shape=[128])
# We can just use 'c.eval()' without passing 'sess'
print(hhh.eval()==jjj.eval())
sess.close()

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]


C:\Users\aleksander.molak\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [51]:
# Define sample plotter
def plot_smpl(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=.05, hspace=.05)
    
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
        
    return fig

In [38]:
# Define the generator initial params 

Z = tf.placeholder(tf.float32, shape=[None, 100], name='Z')

G_W1 = tf.Variable(xavier_init([100, 128]), name='G_W1')
G_b1 = tf.Variable(tf.zeros(shape=[128]), name='G_b1')

G_W2 = tf.Variable(xavier_init([128, 784]), name='G_W2')
G_b2 = tf.Variable(tf.zeros(shape=[784]), name='G_b2')

theta_G = [G_W1, G_W2, G_b1, G_b2]


# Define the discriminator initial params

X = tf.placeholder(tf.float32, shape=[None, 784], name='X')

D_W1 = tf.Variable(xavier_init([784, 128]), name='D_W1')
D_b1 = tf.Variable(tf.zeros(shape=[128]), name='D_b1')

D_W2 = tf.Variable(xavier_init([128, 1]), name='D_W2')
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2')

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [39]:
# Define the generator network

def generator(z):
    
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    
    return G_prob

In [40]:
# Define the discriminator network

def discriminator(x):
    
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    
    return D_prob, D_logit

In [70]:
G_sample = generator(Z)

D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))


D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)

G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)


def sample_Z(m, n):
    return np.random.uniform(-1, 1, size=[m, n])
    
batch_size = 128
Z_dim = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

mnist = input_data.read_data_sets('MNIST/', one_hot=True)

if not os.path.exists('GANs_output/'):
    os.makedirs('GANs_output/')
    
    
i = 0

for itr in range(10000):
    
    if itr % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})
        
        fig = plot_smpl(samples)
        plt.savefig('GANs_output/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)
        
    X_mb, _ = mnist.train.next_batch(batch_size)
    
    _, D_loss_curr = sess.run([D_solver, D_loss]
                              , feed_dict={X: X_mb, Z: sample_Z(batch_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(batch_size, Z_dim)})
    
    if itr % 1000 == 0:
        print(f'Iter: {itr}')
        print(f'D_loss: {D_loss_curr:.4f}')
        print(f'G_loss: {G_loss_curr:.4f}\n')

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
Iter: 0
D_loss: 1.2772
G_loss: 2.5200

Iter: 1000
D_loss: 0.0081
G_loss: 10.2926

Iter: 2000
D_loss: 0.0456
G_loss: 5.3468

Iter: 3000
D_loss: 0.0936
G_loss: 5.0755

Iter: 4000
D_loss: 0.1106
G_loss: 6.0448

Iter: 5000
D_loss: 0.2968
G_loss: 4.7271

Iter: 6000
D_loss: 0.1872
G_loss: 5.1478

Iter: 7000
D_loss: 0.5437
G_loss: 4.2654

Iter: 8000
D_loss: 0.3591
G_loss: 3.7505

Iter: 9000
D_loss: 0.6694
G_loss: 3.2849



In [56]:
tf.__version__

'1.9.0'

### ADD: tf playground

In [77]:
sess = tf.InteractiveSession()
tf.less(3,5).eval()
sess.close()

C:\Users\aleksander.molak\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
